### Imporing dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re # regular expression

# nltk: natural language toolkit
from nltk.corpus import stopwords # stopwords: the words which don't add any value
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer # to convert text into numbers

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### Downloading Stop words

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\atkar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [4]:
# Total Stop words
len(stopwords.words('english'))

198

---

## Data Preprocessing

In [5]:
# Loading the dataset
df = pd.read_csv('fake_news.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [7]:
# Checking Missing Values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# Replacing null values with empty strings
df = df.fillna('')

In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
# Merging Author name and Title (as it will be the input data for model)
df['content'] = df['author'] + ' ' + df['title']

In [11]:
df['content'].head(5)

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object

In [12]:
# X = df.drop(columns='label', axis=1)
# Y = df['label']

# X.ndim, Y.ndim

In [13]:
# print(X), print(Y)

---

## Stemming
Stemming is a text preprocessing technique that reduces words to their root form.

```
example: actor, actress, acting --> (after stemming) --> act
```

In [14]:
port_stem = PorterStemmer()

In [15]:
def stemming(content):
    # Remove characters other than a-z and A-Z and replace them with spaces
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    # Convert the text to lowercase
    stemmed_content = stemmed_content.lower()
    # Split the text into words
    stemmed_content = stemmed_content.split()
    # Apply stemming to each word and remove stop words
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Join the stemmed words back into a string
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [16]:
# Applying the 'stemming' function on the 'content' column of our dataset
df['content'] = df['content'].apply(stemming)

In [17]:
df['content'].head(4)

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
Name: content, dtype: object

In [18]:
df['content'].values

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], shape=(20800,), dtype=object)

In [19]:
df['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [20]:
# Actual spliting of data and label (for our model)
X = df['content'].values
Y = df['label'].values

X.ndim, Y.ndim

(1, 1)

In [21]:
print(X) , print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


(None, None)

In [22]:
# TfidfVertorizer

# TfidfVectorizer converts a collection of raw documents to a matrix of TF-IDF features.

# TF (Term Frequency): Measures how frequently a term occurs in a document.
                # A higher TF indicates higher importance within that document.

# IDF (Inverse Document Frequency): Measures how important a term is across the entire corpus (collection of documents).
                # Rare words have a higher IDF score, as they are more informative.

# TF-IDF combines these two:  It gives higher weight to terms that appear frequently in a particular document but rarely in the rest of the corpus.
                # This helps to identify words that are discriminative for a specific document.

# In simpler terms, it helps to represent text data numerically for machine learning algorithms.
                # Words that are common across all documents are down-weighted, and words that are unique to a few documents are given more importance.


In [23]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [24]:
X.ndim

2

In [25]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210687 stored elements and shape (20800, 17128)>
  Coords	Values
  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (2079

---

## Visulazing reletion between X and Y

In [26]:
# plt.figure(figsize=(20,20))
# sns.scatterplot(x=df['content'], y=df['label'])
# plt.show()

---

## Training the model

In [27]:
# split the data into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [28]:
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)

LogisticRegression()

In [29]:
classifier.predict(X_test)

array([1, 0, 1, ..., 1, 1, 0], shape=(4160,))

### Model Evaluation

In [30]:
accuracy_score(classifier.predict(X_train), Y_train)

0.9863581730769231

In [31]:
classifier.score(X_train, Y_train), classifier.score(X_test, Y_test)

(0.9863581730769231, 0.9790865384615385)

In [32]:
confusion_matrix(Y_test, classifier.predict(X_test))

array([[2004,   73],
       [  14, 2069]])

In [33]:
classification_report(Y_test, classifier.predict(X_test))

'              precision    recall  f1-score   support\n\n           0       0.99      0.96      0.98      2077\n           1       0.97      0.99      0.98      2083\n\n    accuracy                           0.98      4160\n   macro avg       0.98      0.98      0.98      4160\nweighted avg       0.98      0.98      0.98      4160\n'

---

## Sigmoid Function

In [34]:
# plt.figure(figsize=(20,20))
# sns.scatterplot(x=df['content'], y=df['label'])
# sns.lineplot(x=df['content'], y=classifier.predict(X), color='red')
# plt.show()

---

## Making Predictive System

In [35]:
title = "House Dem Aide: We Didnâ€™t Even See Comeyâ€™s Letter Until Jason Chaffetz Tweeted It"
author = "Darrell Lucus"
text = "dosent matter, as our model is not trained on this data"

content = title + " " + author
content = stemming(content)

content = vectorizer.transform([content])

prediction = classifier.predict(content)
print(prediction)

if (prediction[0]==0):
  print('The news is Realiable')
else:
  print('The news is Unrealiable')

[1]
The news is Unrealiable


---